## Set up the environment

In [1]:
import sagemaker as sage
from sagemaker import get_execution_role

# role = get_execution_role()
# or specify role arn e.g.
role = 'arn:aws:iam::[your aws account]:role/[your sagemaker role]'

sagemaker_session = sage.Session()
model_package_arn='arn:aws:sagemaker:[region]:[your aws account id]:model-package/contract-clause-extraction'


pandas failed to import. Analytics features will be impaired or broken.


## Realtime Inference

In [2]:
from sagemaker import ModelPackage
model_name='contractai'
model = ModelPackage(name=model_name,
                     role=role,
                     model_package_arn=model_package_arn,
                     sagemaker_session=sagemaker_session)

In [3]:
endpoint_name='contractai-edp'

In [4]:
predictor = model.deploy(1, 'ml.p2.xlarge', endpoint_name=endpoint_name)

In [5]:
# connect to the endpoint
predictor = sage.predictor.RealTimePredictor(
    endpoint_name,
    sagemaker_session=sagemaker_session
)

In [6]:
import s3fs
fs = s3fs.S3FileSystem()

with fs.open(f's3://[your s3 path]/input/input.docx') as f:    
    docByteArr = f.read()
    response = predictor.predict(docByteArr)
    result=response.decode('utf-8')
    print(result)

In [7]:
# Delete endpoint
predictor.delete_endpoint()

## Batch Transform Job

In [8]:
s3_output_path='s3://[your s3 path]/output/'
transformer = sage.transformer.Transformer(
    base_transform_job_name='contractai-batch-transform',
    model_name=model_name,
    instance_count=1,
    instance_type='ml.m5.large',
    output_path=s3_output_path,
    sagemaker_session=sagemaker_session
)


In [9]:
transformer.transform(
    's3://[your s3 path]/input/input.docx'
)

# wait until prediction job is completed
transformer.wait()


#### Inspect the Batch Transform Output in S3

In [10]:
s3_client = sage.s3.S3Downloader
s3_client.read_file('s3://[your s3 path]/output/input.docx.out')